In [ ]:
cd /content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data

/content/drive/MyDrive/[Projects]/AI Hub/Pose Detection 기반 실시간 낙상 감지 시스템 개발/Data


In [ ]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('output_train_added.csv')
valid_df = pd.read_csv('output_val_added.csv')

In [ ]:
train_df.head()

,file,frame,label,nose_x,nose_y,nose_score,left_eye_x,left_eye_y,left_eye_score,right_eye_x,...,right_knee_score,left_ankle_x,left_ankle_y,left_ankle_score,right_ankle_x,right_ankle_y,right_ankle_score,mean_joint_velocity,mean_angular_velocity,com_velocity
0,02077_H_A_N_C1.mp4,0,0,433.201904,137.339722,0.123687,433.536316,133.542877,0.128915,435.075867,...,0.983481,459.491669,322.209106,0.983306,476.900970,311.007446,0.971570,0.000000,0.000000,0.000000
1,02077_H_A_N_C1.mp4,1,0,432.706665,139.028076,0.119133,432.644653,135.262482,0.107487,434.446259,...,0.984615,457.475311,320.276947,0.979955,477.392151,310.526978,0.972040,2.094620,0.061670,2.002386
2,02077_H_A_N_C1.mp4,2,0,432.811005,140.320831,0.082605,432.059906,136.109604,0.069186,433.391357,...,0.983896,456.548462,320.828400,0.976981,473.975281,310.356018,0.968025,1.985577,0.032278,1.257726
3,02077_H_A_N_C1.mp4,3,0,431.418213,141.639893,0.084149,430.199036,137.462433,0.066609,431.576538,...,0.986282,456.372253,320.752991,0.978816,474.933746,308.751160,0.972351,2.086352,0.025542,0.676110
4,02077_H_A_N_C1.mp4,4,0,425.485809,142.749344,0.093535,424.375854,138.612152,0.070890,425.632202,...,0.987877,456.317413,321.318939,0.978681,476.850220,307.429901,0.975985,2.901474,0.068981,1.580504


In [ ]:
valid_df.head()

,file,frame,label,nose_x,nose_y,nose_score,left_eye_x,left_eye_y,left_eye_score,right_eye_x,...,right_knee_score,left_ankle_x,left_ankle_y,left_ankle_score,right_ankle_x,right_ankle_y,right_ankle_score,mean_joint_velocity,mean_angular_velocity,com_velocity
0,02366_H_A_SY_C7.mp4,0,0,713.038818,227.906616,0.888511,717.906677,223.113281,0.924195,712.661621,...,0.880025,731.935120,442.980225,0.940621,733.062866,437.023102,0.809121,0.000000,0.000000,0.000000
1,02366_H_A_SY_C7.mp4,1,0,713.864014,227.872955,0.891030,718.779541,223.278336,0.923748,713.481201,...,0.885203,732.559082,442.147339,0.941275,729.728516,437.006561,0.816043,1.657754,0.029526,1.802235
2,02366_H_A_SY_C7.mp4,2,0,713.848267,228.159546,0.878212,718.976013,223.683899,0.919033,714.304138,...,0.852213,729.944092,440.952271,0.927905,731.427673,435.665894,0.765373,3.047907,0.071306,1.078983
3,02366_H_A_SY_C7.mp4,3,0,714.660950,228.523041,0.877201,720.220764,224.102509,0.916390,714.938904,...,0.850723,730.536621,440.957397,0.929609,731.798218,435.709595,0.765879,1.658001,0.030195,0.179968
4,02366_H_A_SY_C7.mp4,4,0,714.568359,229.067261,0.866399,720.067688,224.768997,0.911367,715.088623,...,0.835380,729.281128,442.915710,0.924214,731.149902,437.982208,0.744962,1.822419,0.021735,1.480708


## **Threshold 방법**

In [ ]:
# Threshold 방법 1️ : Global Percentile
# 전체 데이터 분포에서 상위 p% 이상
def apply_threshold_global(df, score_col, percentile=95):
    th = np.percentile(df[score_col], percentile)

    near_fall = (
        (df['label'] == 0) &
        (df[score_col] >= th)
    ).astype(int)

    return near_fall, th

In [ ]:
# Threshold 방법 2 : Per-file Adaptive Threshold
# 사람 / 영상마다 움직임 스케일이 다름
# 파일 단위로 상위 p% 적용
def apply_threshold_per_file(df, score_col, percentile=95):
    near_fall = np.zeros(len(df))

    for file in df['file'].unique():
        sub = df[df['file'] == file]
        th = np.percentile(sub[score_col], percentile)

        idx = sub.index[
            (sub['label'] == 0) &
            (sub[score_col] >= th)
        ]
        near_fall[idx] = 1

    return near_fall

## **Method A**

절대값 기반

In [ ]:
# angular velocity → 낙상 전 가장 직접적인 불안정 신호
# joint velocity / COM velocity → 보조 지표
# 합 = 1.0 (해석 용이)
W_METHOD_A = {
    'mean_joint_velocity': 0.3,
    'mean_angular_velocity': 0.4,
    'com_velocity': 0.3
}

In [ ]:
def compute_instability_method_A(df, weights):
    score = (
        weights['mean_joint_velocity']   * df['mean_joint_velocity'] +
        weights['mean_angular_velocity'] * df['mean_angular_velocity'] +
        weights['com_velocity']          * df['com_velocity']
    )
    return score

train_df['instability_A'] = compute_instability_method_A(train_df, W_METHOD_A)

In [ ]:
train_df['near_fall_A_T1'], th_A_T1 = apply_threshold_global(
    train_df,
    score_col='instability_A',
    percentile=95
)

In [ ]:
train_df['near_fall_A_T2'] = apply_threshold_per_file(
    train_df,
    score_col='instability_A',
    percentile=95
)

In [ ]:
print("A + T1 near-fall count:", train_df['near_fall_A_T1'].sum())
print("A + T2 near-fall count:", train_df['near_fall_A_T2'].sum())

A + T1 near-fall count: 13010
A + T2 near-fall count: 14781.0


## **Method B**

변화량 기반

In [ ]:
for col in ["mean_joint_velocity", "mean_angular_velocity", "com_velocity"]:
    dcol = f"delta_{col}"
    train_df[dcol] = train_df[col].diff().fillna(0).abs()

In [ ]:
W_METHOD_B = {
    'delta_mean_joint_velocity': 0.3,
    'delta_mean_angular_velocity': 0.4,
    'delta_com_velocity': 0.3
}

In [ ]:
def compute_instability_method_B(df, weights):
    score = (
        weights['delta_mean_joint_velocity']   * df['delta_mean_joint_velocity'] +
        weights['delta_mean_angular_velocity'] * df['delta_mean_angular_velocity'] +
        weights['delta_com_velocity']          * df['delta_com_velocity']
    )
    return score

train_df['instability_B'] = compute_instability_method_B(train_df, W_METHOD_B)

In [ ]:
train_df['near_fall_B_T1'], th_B_T1 = apply_threshold_global(
    train_df,
    score_col='instability_B',
    percentile=95
)

In [ ]:
train_df['near_fall_B_T2'] = apply_threshold_per_file(
    train_df,
    score_col='instability_B',
    percentile=95
)

In [ ]:
print("B + T1 near-fall count:", train_df['near_fall_B_T1'].sum())
print("B + T2 near-fall count:", train_df['near_fall_B_T2'].sum())

B + T1 near-fall count: 16583
B + T2 near-fall count: 17263.0


## **Method C**

Window 기반

In [ ]:
WINDOW = 5

for col in ["mean_joint_velocity", "mean_angular_velocity", "com_velocity"]:
    train_df[f"{col}_wmean"] = train_df[col].rolling(WINDOW).mean()

In [ ]:
W_METHOD_C = {
    'mean_joint_velocity_wmean': 0.3,
    'mean_angular_velocity_wmean': 0.4,
    'com_velocity_wmean': 0.3
}

In [ ]:
def compute_instability_method_C(df, weights):
    score = (
        weights['mean_joint_velocity_wmean']   * df['mean_joint_velocity_wmean'] +
        weights['mean_angular_velocity_wmean'] * df['mean_angular_velocity_wmean'] +
        weights['com_velocity_wmean']          * df['com_velocity_wmean']
    )
    return score

train_df['instability_C'] = compute_instability_method_C(train_df, W_METHOD_C)

In [ ]:
train_df['near_fall_C_T1'], th_C_T1 = apply_threshold_global(
    train_df,
    score_col='instability_C',
    percentile=95
)

In [ ]:
train_df['near_fall_C_T2'] = apply_threshold_per_file(
    train_df,
    score_col='instability_C',
    percentile=95
)

In [ ]:
print("C + T1 near-fall count:", train_df['near_fall_C_T1'].sum())
print("C + T2 near-fall count:", train_df['near_fall_C_T2'].sum())

C + T1 near-fall count: 0
C + T2 near-fall count: 14181.0


## **Method 별 비교**

- C1. 선행성 : Y(낙상) 이전에 등장해야 함
- C2. 근접성 : Y와 너무 멀면 의미 없음
- C3. 비침투성 : 정상 N 구간을 과도하게 침범하면 안 됨

In [ ]:
# Pre-Fall Coverage
# 각 낙상 이벤트(Y) 마다 Y 직전 T 프레임 안에 near-fall이 1개 이상 있으면 성공
def get_fall_events(df):
    fall_events = []

    for file in df['file'].unique():
        sub = df[df['file'] == file].sort_values('frame')
        fall_frames = sub.loc[sub['label'] == 1, 'frame'].values

        for f in fall_frames:
            fall_events.append((file, f))

    return fall_events

def compute_pre_fall_coverage(df,near_fall_col,T=20):
    fall_events = get_fall_events(df)
    covered = 0

    for file, fall_frame in fall_events:
        sub = df[df['file'] == file]

        window = sub[
            (sub['frame'] < fall_frame) &
            (sub['frame'] >= fall_frame - T)
        ]

        if window[near_fall_col].sum() > 0:
            covered += 1

    coverage = covered / len(fall_events) if len(fall_events) > 0 else 0
    return coverage

In [ ]:
# Lead Time
# Y 발생 프레임 − 가장 가까운 near-fall 프레임 (단, Y 이전 것만)
def compute_lead_times(
    df,
    near_fall_col,
    T=50
):
    fall_events = get_fall_events(df)
    lead_times = []

    for file, fall_frame in fall_events:
        sub = df[df['file'] == file]

        window = sub[
            (sub['frame'] < fall_frame) &
            (sub['frame'] >= fall_frame - T) &
            (sub[near_fall_col] == 1)
        ]

        if len(window) > 0:
            nearest_nf_frame = window['frame'].max()
            lead_times.append(fall_frame - nearest_nf_frame)

    return np.array(lead_times)

In [ ]:
# Non-intrusiveness
# 정상(N) 구간에서 너무 자주 울리면 안 됨
# 즉, 쓸데없는 near-fall 알람 비율 이 낮아야 함
def compute_non_intrusiveness(df, near_fall_col):
    normal = df[df['label'] == 0]

    if len(normal) == 0:
        return None

    false_nf_rate = normal[near_fall_col].sum() / len(normal)
    non_intrusive = 1 - false_nf_rate

    return non_intrusive

In [ ]:
# 요약 통계
def summarize_lead_time(lead_times):
    if len(lead_times) == 0:
        return {
            'mean': None,
            'median': None,
            'q25': None,
            'q75': None
        }

    return {
        'mean':   np.mean(lead_times),
        'median': np.median(lead_times),
        'q25':    np.percentile(lead_times, 25),
        'q75':    np.percentile(lead_times, 75)
    }

In [ ]:
def evaluate_near_fall_method(
    df,
    near_fall_col,
    coverage_T=20,
    lead_T=50
):
    coverage = compute_pre_fall_coverage(
        df,
        near_fall_col=near_fall_col,
        T=coverage_T
    )

    lead_times = compute_lead_times(
        df,
        near_fall_col=near_fall_col,
        T=lead_T
    )
    lead_summary = summarize_lead_time(lead_times)

    non_intrusive = compute_non_intrusiveness(
        df,
        near_fall_col=near_fall_col
    )

    return {
        'Coverage@{}'.format(coverage_T): coverage,
        'LeadTime_mean':   lead_summary['mean'],
        'LeadTime_median': lead_summary['median'],
        'LeadTime_q25':    lead_summary['q25'],
        'LeadTime_q75':    lead_summary['q75'],
        'NonIntrusive':    non_intrusive
    }

In [ ]:
METHODS = {
    'A_T1': 'near_fall_A_T1',
    'A_T2': 'near_fall_A_T2',
    'B_T1': 'near_fall_B_T1',
    'B_T2': 'near_fall_B_T2',
    'C_T1': 'near_fall_C_T1',
    'C_T2': 'near_fall_C_T2',
}

results = []

for name, col in METHODS.items():
    metrics = evaluate_near_fall_method(
        train_df,
        near_fall_col=col,
        coverage_T=20,
        lead_T=50
    )

    metrics['Method'] = name
    results.append(metrics)

result_df = pd.DataFrame(results)

In [ ]:
# A_T1 선정 이유
# Coverage	낙상 전조를 가장 많이 잡음
# Lead Time	평균 14프레임 → 충분한 반응 시간
# NonIntrusive	정상 구간 방해 최소
# 단순성	weight 고정 + global threshold
# 재현성	파일 간 편차에 덜 민감
result_df = result_df[
    ['Method',
     'Coverage@20',
     'LeadTime_mean',
     'LeadTime_median',
     'NonIntrusive']
].sort_values(
    by=['Coverage@20', 'NonIntrusive'],
    ascending=False
)

result_df

,Method,Coverage@20,LeadTime_mean,LeadTime_median,NonIntrusive
0,A_T1,0.329172,13.722433,11.0,0.967028
3,B_T2,0.307041,19.137907,15.0,0.956249
1,A_T2,0.291030,15.749280,13.0,0.962540
2,B_T1,0.288778,19.117429,15.0,0.957973
5,C_T2,0.127105,17.774005,14.0,0.964060
4,C_T1,0.000000,NaN,NaN,1.000000
